<h4>18mca029 & 19mca162 <h4> 
    
<h1> BDA - SPARK PROJECT - Churn Prediction <h1>

<b>First Of all we will understand that what is Churn and Where this Churn Prediction system will helps. </b>


 ->  Churn prediction is knowing which users are going to stop using your platform in the future.
 
 ->  These predictions are used by Marketers to proactively take retention actions on Churning users. For example, If Amazon knew a segment of users who were at risk of Churning (Not ordering) on the platform then Amazon can proactively engage these users with Offers etc. and help them make a purchase.


First We have download import pyspark 

After That we have given Name of the Project as  BDA_PROJECT using appName()

In [2]:
import findspark
findspark.init()
import pyspark

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('BDA_PROJECT').getOrCreate()

Logistic Regression is a non Linear Regression which will helps to find non - linear relationship betweeen features and output(Churn).

Logistic Regression is used for both Regression and classification  

Using MLlib we have used Machine Learning Algorithm named Logistic Regression

In [4]:
from pyspark.ml.classification import LogisticRegression

For this project we have taken a static dataset of Telco Ccustomer Churn  from Kaggle 

In [5]:
df=spark.read.csv('WA_Fn-UseC_-Telco-Customer-Churn.csv',header = True, inferSchema = True)

printSchema is used to see the list of  data columns and their datatype of Dataset
As per the output of printschema we can say that 
in this dataset there are 21 columns and thier datatypes

In [6]:
df.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: string (nullable = true)
 |-- Churn: string (nullable = true)



There are total 7043 different customers of data 

In [7]:
df.count()

7043

We have droped out CustomerID column because it is not helps to predict something.

In [8]:
df = df.drop("customerID")

If We will analyse datatypes of every column then we will find so many problems in the dataset
like

Most of the data is in string format and ml never accept any string value as a input

so we have to change data and datatypes as per the requirements


In [31]:
df.select("TotalCharges_t").distinct().show(10,False)

+--------------+
|TotalCharges_t|
+--------------+
|1874.45       |
|1752.65       |
|244.8         |
|673.25        |
|4326.8        |
|229.6         |
|936.7         |
|5388.15       |
|168.5         |
|49.8          |
+--------------+
only showing top 10 rows



here we have seen that data is entered as a string like '49.8' 
so
we just have to change datatype

In [9]:
df = df.withColumn("TotalCharges_t", df['TotalCharges'].cast('double')).drop('TotalCharges')

and also so many columns have string data like 'Yes' and 'No'

and we will change this data in binary using standard method 

In [10]:
from pyspark.ml.feature import StringIndexer

In [11]:
indexer = StringIndexer(inputCol = "Churn", outputCol = "Churn_c")
df = indexer.fit(df).transform(df)
indexer1 = StringIndexer(inputCol = "gender", outputCol = "Gender_g")
df = indexer1.fit(df).transform(df)
indexer2 = StringIndexer(inputCol = "Partner", outputCol = "Partner_p")
df = indexer2.fit(df).transform(df)
indexer3 = StringIndexer(inputCol = "Dependents", outputCol = "Dependents_d")
df = indexer3.fit(df).transform(df)
indexer4 = StringIndexer(inputCol = "OnlineSecurity", outputCol = "OnlineSecurity_o")
df = indexer4.fit(df).transform(df)

df = df.drop("Churn")
df = df.drop("gender")
df = df.drop("Partner")
df = df.drop("Dependents")
df = df.drop("OnlineSecurity")

df.show()

+-------------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+--------------+-------+--------+---------+------------+----------------+
|SeniorCitizen|tenure|PhoneService|   MultipleLines|InternetService|       OnlineBackup|   DeviceProtection|        TechSupport|        StreamingTV|    StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges_t|Churn_c|Gender_g|Partner_p|Dependents_d|OnlineSecurity_o|
+-------------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+--------------+-------+--------+---------+------------+----------------+
|            0|     1|          No|No phone service|            DSL|     

We will assign features with VectorAssembler 

In [12]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [16]:
assembler = VectorAssembler (inputCols = ['SeniorCitizen','tenure','MonthlyCharges','TotalCharges_t','OnlineSecurity_o','Gender_g',
                                        'Partner_p','Dependents_d' ],outputCol = 'features')

and features have transformed in current DataFrame

In [17]:
output_data = assembler.transform(df)

So , here we can see that features column is also transformed in currunt Dataframe

features column is in the form of vector 

In [18]:
output_data.printSchema()

root
 |-- SeniorCitizen: integer (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges_t: double (nullable = true)
 |-- Churn_c: double (nullable = false)
 |-- Gender_g: double (nullable = false)
 |-- Partner_p: double (nullable = false)
 |-- Dependents_d: double (nullable = false)
 |-- OnlineSecurity_o: double (nullable = false)
 |-- features: vector (nullable = true)



Here we can see that how first row of the output_data looks like..

In [19]:
output_data.head(1)

[Row(SeniorCitizen=0, tenure=1, PhoneService='No', MultipleLines='No phone service', InternetService='DSL', OnlineBackup='Yes', DeviceProtection='No', TechSupport='No', StreamingTV='No', StreamingMovies='No', Contract='Month-to-month', PaperlessBilling='Yes', PaymentMethod='Electronic check', MonthlyCharges=29.85, TotalCharges_t=29.85, Churn_c=0.0, Gender_g=1.0, Partner_p=1.0, Dependents_d=0.0, OnlineSecurity_o=0.0, features=DenseVector([0.0, 1.0, 29.85, 29.85, 0.0, 1.0, 1.0, 0.0]))]

we have differenciate particular 2 columns features and Churn
to make next processes easy

now in this dataframe there are only 2 types of data 


features that is independent data


and


Churn_c that is dependent on the feature data

In [22]:
final_data = output_data.select('features','Churn_c')
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Churn_c: double (nullable = false)



in  the furthur process we need to find accuracy 

and for that

we will spilt data for 

train the model

and

test model

In [23]:
train,test = final_data.randomSplit([0.8,0.2])

here , we are going to create a model 

In [24]:
model=LogisticRegression(labelCol = 'Churn_c')

and using .fit method we have tried to train our model

In [26]:
model = model.fit(train)

Py4JJavaError: An error occurred while calling o372.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 17.0 failed 1 times, most recent failure: Lost task 0.0 in stage 17.0 (TID 17, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$4: (struct<SeniorCitizen_double_VectorAssembler_3b6bead3ffaa:double,tenure_double_VectorAssembler_3b6bead3ffaa:double,MonthlyCharges:double,TotalCharges_t:double,OnlineSecurity_o:double,Gender_g:double,Partner_p:double,Dependents_d:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.sort_addToSorter_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at

In [4]:
summary = model.summary

NameError: name 'model' is not defined

Thank You.